In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import logging

import cipy

logger = logging.getLogger()
handler = logger.handlers[0]
handler.setLevel(logging.CRITICAL)

conn_creds = cipy.db.get_conn_creds('DATABASE_URL')
pgdb = cipy.db.PostgresDB(conn_creds)

---

## User Management

- create new user accounts and store passwords in a secure format
- delete existing user accounts (along with any owned reviews)
- user login

In [4]:
%run ../scripts/create_user.py --test

Enter user name: Sam C
Enter user email: samc@gmail.com
Confirm user email: samc@gmail.com
Enter password: ········
Confirm password: ········


2016-07-12 22:27:25,784 - create_user - INFO - created user id=3: {'owned_review_ids': None, 'name': 'Sam C', 'email': 'samc@gmail.com', 'created_ts': '2016-07-13T01:48:11.754032Z', 'review_ids': None}


In [5]:
list(pgdb.run_query('SELECT * from users'))

[{'created_ts': datetime.datetime(2016, 7, 7, 17, 56, 7),
  'email': 'burtdewilde@gmail.com',
  'name': 'Burton DeWilde',
  'owned_review_ids': [1],
  'password': '$2a$06$5qfLF4y/sfXkc8XhZ360i.48V5GaQfxF5Uy8zVJcO6dLmUqX9JGie',
  'review_ids': [1],
  'user_id': 1},
 {'created_ts': datetime.datetime(2016, 7, 13, 1, 48, 12),
  'email': 'samc@gmail.com',
  'name': 'Sam C',
  'owned_review_ids': None,
  'password': '$2a$06$MHhP.XjRGfSO7eiDiNLFieJEGoOXxQruc6.Sd3nb.vImvoAuO09x.',
  'review_ids': None,
  'user_id': 3}]

In [2]:
%run ../scripts/login_user.py

Enter email: burtdewilde@gmail.com
Enter password: ········


2016-07-23 10:18:00,069 - login_user - INFO - Welcome, Burton DeWilde id=1


In [2]:
%run ../scripts/delete_user.py --user_id=3 --test

2016-07-12 21:39:29,460 - delete_user - INFO - deleted user id=1 from reviews (TEST)
2016-07-12 21:39:29,460 - delete_user - INFO - deleted user id=1 (TEST)


---

## Review Management

- create new reviews (with user as owner)
- delete existing owned reviews
- invite/uninvite other users to collaborate on existing reviews
- assign other user as owned review's new owner

In [3]:
%run ../scripts/create_review.py --user_id=1 --test

Review name: My Great Review
Review description (optional):
Live long and prosper.


2016-07-24 16:51:58,634 - create_review - INFO - created review (TEST): {'name': 'My Great Review', 'settings': {'num_citation_screening_reviewers': 2, 'required_fulltext_screener_id': None, 'num_fulltext_screening_reviewers': 2, 'required_citation_screener_id': None}, 'user_ids': [1], 'owner_user_id': 1, 'created_ts': '2016-07-24T20:50:39.039925Z', 'description': 'Live long and prosper.'}


In [4]:
list(pgdb.run_query("SELECT * FROM reviews where review_id=1"))[0]

{'created_ts': datetime.datetime(2016, 7, 13, 1, 40, 27),
 'description': 'International policy has sought to emphasize and strengthen the link between the conservation of natural ecosystems and human development. Furthermore, international conservation organizations have broadened their objectives beyond nature-based goals to recognize the contribution of conservation interventions in sustaining ecosystem services upon which human populations are dependent. While many indices have been developed to measure various human well-being domains, the strength of evidence to support the effects, both positive and negative, of conservation interventions on human well-being, is still unclear.\\n\\nThis protocol describes the methodology for examining the research question: What are the impacts of nature conservation interventions on different domains of human well-being in developing countries? Using systematic mapping, this study will scope and identify studies that measure the impacts of natu

In [3]:
%run ../scripts/delete_review.py --user_id=1 --review_id=1 --test

2016-07-12 21:59:45,881 - delete_review - INFO - deleted review id=1 (TEST)


In [37]:
%run ../scripts/manage_collaborators.py --owner_user_id=1 --review_id=1 --add_user_emails "samc@gmail.com"

2016-07-12 22:40:52,400 - manage_collaborators - INFO - user id=3 added as collaborator to review id=1 


In [38]:
print('review:', list(pgdb.run_query('SELECT review_id, user_ids FROM reviews WHERE review_id=1')))
print('user:', list(pgdb.run_query('SELECT user_id, review_ids FROM users WHERE user_id=3')))

review: [{'review_id': 1, 'user_ids': [1, 3]}]
user: [{'user_id': 3, 'review_ids': [1]}]


In [39]:
%run ../scripts/manage_collaborators.py --owner_user_id=1 --review_id=1 --remove_user_emails "samc@gmail.com"

2016-07-12 22:40:55,110 - manage_collaborators - INFO - user id=3 removed as collaborator to review id=1 


In [40]:
print('review:', list(pgdb.run_query('SELECT review_id, user_ids FROM reviews WHERE review_id=1')))
print('user:', list(pgdb.run_query('SELECT user_id, review_ids FROM users WHERE user_id=3')))

review: [{'review_id': 1, 'user_ids': [1]}]
user: [{'user_id': 3, 'review_ids': []}]


---

## Review Planning

- facilitate systematic review planning while also gathering structured data that informs and is informed by the citation pre-screening process; user entry of the following fields:
    - objective
    - research questions, ranked
    - PICO statements
    - grouped keyterms (with automatic boolean search query generation)
    - selection criteria, with shorthand labels
- automatically generate boolean search queries from given keyterms
- after enough citations have been screened, suggest good/bad keyterms for search query

In [3]:
%run ../scripts/plan_review.py --user_id=1 --review_id=1 --test


PROJECT PLAN

Objective:
To assess and characterize the current state and distribution of the existing evidence base around the causal linkages between both positive and negative effects of nature conservation and human well-being.

Research Questions:
 0 What are the impacts of nature conservation interventions on different domains of human well-being in developing countries?
 1 What is the current state and distribution of evidence?
 2 What types of impacts from conservation interventions on human well-being are measured?
 3 What types of ecosystem services are explicitly associated with the impacts of conservation interventions on human well-being?
 4 What populations are affected by conservation and/ or focus of studies?
 5 How does the evidence base align with major priorities and investments of implementing agencies?

PICO:
- Population    : Human populations, including individuals, households, communities or nation states in non-OECD countries
- Intervention  : Adoption or impl

2016-07-23 11:10:41,502 - plan_review - INFO - valid record: review_id=1 with {'pico', 'selection_criteria', 'objective', 'research_questions', 'keyterms'}, (TEST)



PROJECT PLAN

Objective:
To assess and characterize the current state and distribution of the existing evidence base around the causal linkages between both positive and negative effects of nature conservation and human well-being.

Research Questions:
 0 What are the impacts of nature conservation interventions on different domains of human well-being in developing countries?
 1 What is the current state and distribution of evidence?
 2 What types of impacts from conservation interventions on human well-being are measured?
 3 What types of ecosystem services are explicitly associated with the impacts of conservation interventions on human well-being?
 4 What populations are affected by conservation and/ or focus of studies?
 5 How does the evidence base align with major priorities and investments of implementing agencies?

PICO:
- Population    : Human populations, including individuals, households, communities or nation states in non-OECD countries
- Intervention  : Adoption or impl

In [5]:
query = "SELECT keyterms FROM review_plans WHERE review_id = 1"
keyterms = list(pgdb.run_query(query))[0]['keyterms']
print(cipy.utils.get_boolean_search_query(keyterms))

(("wellbeing" OR "well-being" OR "well being") OR ("ecosystem service" OR "ecosystem services") OR "nutrition" OR ("skill" OR "skills") OR ("empower" OR "empowering") OR ("clean water" OR "livelihood") OR ("livelihoods" OR "food security") OR ("resilience" OR "vulnerability") OR ("capital" OR "social capital") OR ("attitude" OR "attitudes") OR ("perception" OR "perceptions") OR ("health" OR "human health") OR ("human capital" OR "knowledge") OR "traditional knowledge")
AND
(("marine" OR "freshwater") OR "coastal" OR ("forest" OR "forests" OR "forestry") OR ("ecosystem" OR "ecosystems") OR "species" OR ("habitat" OR "habitats") OR "biodiversity" OR ("sustainable" OR "sustainability") OR ("ecology" OR "ecological") OR "integrated" OR "landscape" OR "seascape" OR ("coral reef" OR "coral reefs") OR ("natural resources" OR "natural resource"))
AND
(("human" OR "humans" OR "humanity") OR "people" OR ("person" OR "persons") OR ("community" OR "communities") OR ("household" OR "households") OR

---

## Citation Ingestion and De-duplication

- load citations from RIS or BibTex files then parse, standardize, sanitize, validate, and store the data
- identify duplicate citations using a sophisticated model and assign the most complete record in a set of duplicates as the "canonical" record

In [13]:
%run ../scripts/ingest_citations.py --citations ../data/raw/citation_files/phase_2_demo_citations.ris --user_id=1 --review_id=1 --test

2016-07-11 22:24:20,813 - ingest_citations - INFO - parsing records in ../data/raw/citation_files/phase_2_demo_citations.ris
2016-07-11 22:24:20,814 - ingest_citations - INFO - valid record: Ecological protection and well-being, 2013
2016-07-11 22:24:20,815 - ingest_citations - INFO - valid record: The economic value of forest ecosystems, 2001
2016-07-11 22:24:20,816 - ingest_citations - INFO - valid record: Contribution of tourism development to protected area management: Local stakeholder perspectives, 2009
2016-07-11 22:24:20,817 - ingest_citations - INFO - 3 valid records inserted into appname db (TEST)


In [17]:
num_citations = list(pgdb.run_query('SELECT COUNT(1) FROM citations WHERE review_id = 1'))[0]['count']
print('total # citations =', num_citations)

total # citations = 28709


In [7]:
%run ../scripts/dedupe_records.py --review_id=1 --threshold=auto --settings=../models/dedupe_citations_settings --test

2016-07-11 22:19:58,881 - dedupe_records - INFO - reading dedupe settings from ../models/dedupe_citations_settings
2016-07-11 22:20:19,236 - dedupe_records - INFO - duplicate threshold = 0.827943
2016-07-11 22:20:19,879 - dedupe_records - INFO - found 361 duplicate clusters
2016-07-11 22:20:20,020 - dedupe_records - INFO - inserted 726 records into duplicates db (TEST)


In [2]:
query = """
SELECT canonical_citation_id, array_agg(citation_id) AS citation_ids, AVG(duplicate_score) AS avg_score
FROM duplicates
GROUP BY 1 HAVING AVG(duplicate_score) > 0.9 ORDER BY 1 ASC
LIMIT 1
"""
dupes = list(pgdb.run_query(query))[0]
print('citations {} are duplicates with avg. duplicate score = {}'.format(
        dupes['citation_ids'], round(dupes['avg_score'], 6)))

query = """
SELECT citation_id, authors, title, abstract, publication_year, keywords
FROM citations
WHERE citation_id = ANY(%(citation_ids)s)
"""
for record in pgdb.run_query(query, {'citation_ids': dupes['citation_ids']}):
    cipy.utils.present_citation(record)

citations [15, 14] are duplicates with avg. duplicate score = 0.999982

TITLE:    Managing the Brisbane River and Moreton Bay: an integrated research/management program to reduce impacts on an Australian estuary
YEAR:     2001
AUTHORS:  Abal, E G; Dennison, W C; Greenfield, P F
ABSTRACT: The Brisbane River and Moreton Bay Study, an interdisciplinary study of Moreton Bay and its major tributaries, was initiated to address water quality issues which link sewage and diffuse loading with environmental degradation. Runoff and deposition of fine-grained sediments into Moreton Bay, followed by resuspension, have been linked with increased turbidity and significant loss of seagrass habitat. Sewage-derived nutrient enrichment, particularly nitrogen (N), has been linked to algal blooms by sewage plume maps. Blooms of a marine cyanobacterium, Lyngbya majuscula, in Moreton Bay have resulted in significant impacts on human health (e.g., contact dermatitis) and ecological health (e.g., seagrass loss

---

## Initial Ranking of Citations

- sample citations ranked by overlap with keyterms; user pre-screens citations until 10 have been included and 10 have been excluded
- based on included/excluded citations, rank citations by ratio of relevant to irrelevant keyterms and present those most likely to be relevant to the user for pre-screening

In [2]:
%run ../scripts/screen_citations.py --user_id=1 --review_id=1 --auto

2016-07-24 23:16:38,917 - screen_citations - INFO - 0 citations included, 0 citations excluded



TITLE:    Marine ecological capital: Concept and attributes
YEAR:     2010.0
AUTHORS:  Chen, S; Du, G; Ke, S; Li, J; Ren, D; Wang, D; Wang, L; Wang, M; Wang, Q; Xia, T; Zhao, Z
ABSTRACT: Marine ecological resources are the foundation of social and economic development. A nation's marine economic development mainly depends on the quantity and quality of marine ecological resources it possesses. Marine ecological resources, as the important input to marine industry, can be viewed as "capital". According to the World Bank's definition, capital consists of natural capital, man-made capital, human capital and social capital. Marine ecological resources are the important component of natural capital. Valuation and capitalized management of marine ecological resources are very important for human's sustainable development. This article proposed the concept of marine ecological capital based on the theories of both natural capital and ecosystem services. At the same time the authors consider 

2016-07-24 23:16:47,613 - screen_citations - INFO - 0/10 included, 1/10 excluded



INCLUDE (y/n/u)? n

TITLE:    Trends in development of coastal area management in tropical countries: From central to community orientation
YEAR:     1997.0
AUTHORS:  Christie, P; White, A T
ABSTRACT: The development phases of coastal management in tropical countries are described. Precolonial, centralized, community based, and collaborative coastal management are identified as forms of management which have evolved to what today is called integrated coastal management. Centralized management began during colonial times when any attempt at management was orchestrated from the top of government. Community based coastal management, in response to the failures of more centralized approaches, began in the Philippines through several projects, the experience of which spread to other countries in Southeast Asia and to Ecuador in the 1980s. The concept of collaborative management is now accepted as a description of variations in joint management arrangements between government and community 

2016-07-24 23:16:47,868 - screen_citations - INFO - 0/10 included, 2/10 excluded


...


INCLUDE (y/n/u)? n

TITLE:    Ecotourism and biodiversity conservation in Jozani-Chwaka Bay National Park, Zanzibar
YEAR:     2009.0
AUTHORS:  Salum, L A
ABSTRACT: This study is based on a study undertaken to assess how ecotourism has influenced biodiversity conservation in Jozani-Chwaka Bay National Park (JCBNP). It involved two communities surrounding the park, namely Pete and Kitogani. Field data were collected using structured questionnaires, key informant interviews and field observations. Questionnaires were administered to 76 households, whereas key informant interviews were conducted with foresters and JCBNP officials. Data analysis was undertaken using standard statistical methods. Findings from the study show that biodiversity management in the JCBNP has improved considerably after the introduction of ecotourism. The number of endemic colobus monkeys (Piliocolobus kirkii) and other rare species has increased. However, this achievement has been attained by restricting surroun

2016-07-24 23:17:03,612 - screen_citations - INFO - 9/10 included, 55/10 excluded



INCLUDE (y/n/u)? n

TITLE:    Livelihood Sustainability and Community Based Co-Management of Forest Resources in China: Changes and Improvement
YEAR:     2012.0
AUTHORS:  Chen, H Y; Maddox, D; Shivakoti, G; Zhu, T
ABSTRACT: Community-based co-management (CBCM) has been applied in some communities near natural reserves in China. This paper uses Gansu Baishuijiang National Natural Reserve in China as a case study for livelihood improvements under CBCM projects. We demonstrate change from 2006 to 2010 in five classes of livelihood capital (social, human, natural, physical and financial capitals), illustrating the effectiveness of CBCM projects. Specifically, there are increases in mean family income and improvements in forest conservation. However, some problems in the design and implementation of CBCM projects remain, including the complicated social and political relationship between government and community, social exclusion and uneven application of benefits within communities, and t

2016-07-24 23:17:03,867 - screen_citations - INFO - 10/10 included, 55/10 excluded



INCLUDE (y/n/u)? y


---

## Refinement of Search Keyterms

- based on included/excluded citations, create lists of strongly relevant and irrelevant keyterms that can be used to refine initial set of keyterms